In [1]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 35.9 MB/s 
     |████████████████████████████████| 86 kB 5.4 MB/s 
     |████████████████████████████████| 6.6 MB 35.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
! pip install KeyBERT

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.7 MB/s 
     |████████████████████████████████| 232 kB 30.0 MB/s 
     |████████████████████████████████| 51 kB 6.7 MB/s 
     |████████████████████████████████| 1.2 MB 48.4 MB/s 
  Created wheel for KeyBERT: filename=keybert-0.5.1-py3-none-any.whl size=21332 sha256=ad7c51bbcc3d87f0096e05e3088d0cd82ddd6981dcfcd454e306b5a5e8451018
  Stored in directory: /root/.cache/pip/wheels/8e/95/c5/f5ceed2a9f9e80bc1a706a10a6fb03d726df7a3dd11800a58b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=5d8d1abc959af081ce4f4ed4989137ec1e636ce2b3b3c050badd0f279ba7f4fc
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built KeyBERT sentence-transformers


In [3]:
import spacy
import en_core_web_sm
import pandas as pd
import nltk
import re
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from keybert import KeyBERT
import json

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
details = "Apple Computers, Inc. was founded on April 1, 1976, by college dropouts Steve Jobs and Steve Wozniak, who brought to the new company a vision of changing the way people viewed computers. Jobs and Wozniak wanted to make computers small enough for people to have them in their homes or offices. Simply put, they wanted a computer that was user-friendly.Jobs and Wozniak started out building the Apple I in Jobs' garage and sold them without a monitor, keyboard, or casing (which they decided to add on in 1977). The Apple II revolutionized the computer industry with the introduction of the first-ever color graphics. Sales jumped from $7.8 million in 1978 to $117 million in 1980, the year Apple went public."

In [8]:
#preprocessing function
def preprocessing(text):
    text = re.sub(r'\n',"",text)
    return text

#preprocessing
pre_details = preprocessing(details)
print(details)

Apple Computers, Inc. was founded on April 1, 1976, by college dropouts Steve Jobs and Steve Wozniak, who brought to the new company a vision of changing the way people viewed computers. Jobs and Wozniak wanted to make computers small enough for people to have them in their homes or offices. Simply put, they wanted a computer that was user-friendly.Jobs and Wozniak started out building the Apple I in Jobs' garage and sold them without a monitor, keyboard, or casing (which they decided to add on in 1977). The Apple II revolutionized the computer industry with the introduction of the first-ever color graphics. Sales jumped from $7.8 million in 1978 to $117 million in 1980, the year Apple went public.


### Summarizing

In [9]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

Downloading:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

In [20]:
summarizor = pipeline('summarization', model = 'facebook/bart-large-cnn')

In [21]:
 z = summarizor(details)

In [23]:
 z[0]['summary_text']

"Jobs and Wozniak started out building the Apple I in Jobs' garage. The Apple II revolutionized the computer industry with the introduction of the first-ever color graphics. Sales jumped from $7.8 million in 1978 to $117 million in 1980, the year Apple went public."

In [24]:
len(tokenizer.encode(details))

161

In [25]:
tokenizer.model_max_length

1024

### Key word extraction

In [27]:
kw_model = KeyBERT()
# key_words = kw_model.extract_keywords(details,keyphrase_ngram_range=(2,2), top_n=3)
# key_words[0][0]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [63]:
class SMS_summarizor(object):
    
    def __init__(self,paragraph):
        self.para = paragraph
        #preprocessing the data
        self.__para1 = self.__preprocessor(self.para)
        
    def __preprocessor(self,text):
        #Extend the preprocessor function as for the requirements
        text = re.sub(r'\n',"",text)
        return text
    
    def summarize_sms(self, max_length,min_length):
        """
        Summarize the given content according the max_length and min_length parameters

        max_length ----> int
        min_length ----> int
        
        """
        #asserting to remove errors and warnings
        assert max_length < len(tokenizer.encode(self.__para1)), "Max length must be smaller than {} ".format(len(tokenizer.encode(self.__para1)))
        assert len(tokenizer.encode(self.__para1)) < tokenizer.model_max_length, "Text document must contain more less information"
        #summarizing the content
        summary = summarizor(self.__para1,max_length=max_length, min_length=min_length)
        self.text = summary[0]['summary_text']
        
    def title_generator(self,number_of_words):
        """
        Generating the tiitle usinng key_word extraction
        number_of_words ---> Define the title of the summary
                              type--->int
        """
        assert type(number_of_words) == int,"number of words must be an integer"
        assert number_of_words <3, "Limit the words to 3"
        key_words = kw_model.extract_keywords(self.__para1,keyphrase_ngram_range=(number_of_words,number_of_words), top_n=3)
        self.title = key_words[0][0]
        
    def save_json(self,path,json_file_name):
        """
        Save the Json file of the summarized content and it's generated title.
        path ---> Path must be ended with \
                    ex-:C:\Lachin\DataSets\SMS\
        json_file_name ---> name.json
        """
  
        assert type(path) == str and type(json_file_name) == str , "Pathn and json_file_name must be string"
        assert json_file_name.endswith("json"),"file_name must be a .json extension"
        dict1 = {
            "SMS_Summarizor":{
                "Title": str(self.title),
                "Summary": str(self.text)
            }
        }
        
        location = open(path + json_file_name,'w')
        json.dump(dict1,location, indent = 2)
        print("Json File successfull saved.")

In [56]:
s1 = SMS_summarizor(details)

In [57]:
s1.summarize_sms(100,70)

In [58]:
s1.text

"Jobs and Wozniak started out building the Apple I in Jobs' garage and sold them without a monitor, keyboard, or casing. The Apple II revolutionized the computer industry with the introduction of the first-ever color graphics. Sales jumped from $7.8 million in 1978 to $117 million in 1980, the year Apple went public."

In [59]:
s1.title_generator(2)

In [60]:
s1.title

'apple computers'

In [61]:
path = "/content/drive/MyDrive/Colab Notebooks/"
name = "sms.json"
s1.save_json(path,name)

Json File successfull saved.
